In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import t
from math import sqrt, floor
import statistics
import pandas as pd
from scipy.stats import shapiro, levene,bartlett

Importamos los datos

In [ ]:
datos = pd.read_csv("Reto1.csv")

grupo_a = datos[datos['Grupo'] == 'A']['Satisfacción']
grupo_b = datos[datos['Grupo'] == 'B']['Satisfacción']

# Mostrar los primeros registros de cada grupo para verificar
print("Grupo A:")
print(grupo_a.head(10))

print("\nGrupo B:")
print(grupo_b.head(10))

Primer Supuesto: las dos poblaciones de distribuyen normal

Para comprobar que nuestros datos provienen de una distribución normal usaremos la prueba de Shapiro Wilk

La prueba de Shapiro-Wilk es una prueba estadística utilizada para evaluar si una muestra de datos sigue una distribución normal o gaussiana. Funciona comparando los valores observados con los valores esperados bajo la hipótesis nula de normalidad.

Hipótesis nula (H0): La muestra proviene de una población con una distribución normal.

Hipótesis alternativa (H1): La muestra no proviene de una población con una distribución normal.

In [ ]:
# Verificar normalidad en el Grupo A
shapiro_test_a = shapiro(grupo_a)
print("Shapiro-Wilk test para Grupo A:", shapiro_test_a)

# Verificar normalidad en el Grupo B
shapiro_test_b = shapiro(grupo_b)
print("Shapiro-Wilk test para Grupo B:", shapiro_test_b)

Shapiro-Wilk test para Grupo A: ShapiroResult(statistic=0.9827495217323303, pvalue=0.6722097396850586)
Shapiro-Wilk test para Grupo B: ShapiroResult(statistic=0.9713166952133179, pvalue=0.26161736249923706)


Usando una significancia del 0,05
podemos ver que para ambas muestras los p-valores son mayores que la significancia. Por lo tanto no hay suficiente evidencia cientifica para rechazar la hipótesis nula, asi que ambas muestras provienen de una distribución normal.

# Segundo Supuesto: Homogeneidad de Varianzas

Para verificar el supuesto que las varianzas de los datos son iguales usaremos el test Bartlett.


La prueba de Bartlett es una prueba estadística utilizada para evaluar si las varianzas de dos o más grupos de datos son iguales. Es una prueba paramétrica que asume que los datos provienen de distribuciones normales.

Hipótesis nula (H0): Las varianzas de todos los grupos son iguales (homogeneidad de varianzas).

Hipótesis alternativa (H1): Al menos una de las varianzas es diferente.

In [ ]:
# Realizar la prueba de Bartlett
estadistica_bartlett, p_valor_bartlett = bartlett(grupo_a,grupo_b)

# Imprimir los resultados
print("Estadística de prueba de Bartlett:", estadistica_bartlett)
print("Valor p de la prueba de Bartlett:", p_valor_bartlett)

# Interpretar los resultados
nivel_significancia = 0.05
if p_valor_bartlett < nivel_significancia:
    print("Las varianzas de los grupos son diferentes (rechazamos H0)")
else:
    print("Las varianzas de los grupos son iguales (no rechazamos H0)")


Estadística de prueba de Bartlett: 5.495874887849253
Valor p de la prueba de Bartlett: 0.019061387916391257
Las varianzas de los grupos son diferentes (rechazamos H0)


Usando una significancia del 0,05 podemos ver que el p-valor es menor que la significancia. Por lo tanto hay suficiente evidencia cientifica para rechazar la hipótesis nula, las varianzas de las muestras son diferentes

 **Todos los supuestos no se cumplen, pero aún asi realizaremos la prueba**

Prueba T con los datos originales

In [ ]:
def prueba_t_student(a,b,n_colas = 2,significancia = 0.05):
  # Calcular la media, el tamaño de la muestra, desviación estandar muestral y los grados de libertad
  media_a = np.mean(a)
  media_b = np.mean(b)
  n_a = len(a)
  n_b = len(b)
  s_a = np.std(a)
  s_b = np.std(b)
  gl = (s_a **2/n_a + s_b **2/n_b)**2 / ((1/(n_a - 1)*(s_a **2/n_a)**2) + ((1/(n_b - 1)*(s_b **2/n_b)**2)))
  #imprimir información importante
  print("Ho = Las muestras vienen de la misma población")
  print("Ha = Las muestras provienen de poblaciones diferentes")
  print(f"Medias muestrales de muestra A = {media_a}, muestra B = {media_b}")
  print(f"Tamaños de la muestra A = {n_a}, muestra B = {n_b}")
  print(f"Desviaciónes Estandar de la muestra A = {s_a}, muestra B = {s_b}")
  print(f"Grados de Libertad = {gl}")

  #
  print(f"x̄1 - x̄2 ~ N(μ^x̄1 - μ^x̄2 = μ1 - μ2,s^x̄1-x̄2 = {sqrt(s_a**2/n_a +s_b**2/n_b )})")
  print("-------------------------------------------------------------------")
  print("Valor Critico :")
  if n_colas == 2:
    # Calculamos los intervalos de confianza
    significancia_dos_colas = round(significancia/2, 3)
    t_valor = t.ppf(1 - significancia_dos_colas, gl)# Calcula el valor crítico de t
    me = abs(t_valor * sqrt(s_a**2/n_a +s_b**2/n_b ))
    d = media_a - media_b
    d_izq = round(d - me,3)
    d_der = round(d + me,3)
  #imprimir información importante
  print("significancia para dos colas = ", significancia_dos_colas)
  print("t_valor = ",t_valor)
  print("me = ",me)
  print("d = ",d)
  print(f"intervalo confianza = [{d_izq},{d_der}]")

  if d_izq < d <d_der:
        print("La diferencia de la media muestral está fuera de la zona de rechazo")
        print("Conclusión --> No hay suficiente evidencia para rechazar la Hipotesis Nula. Las muestras provienen de la misma población")
  else:
        print("La diferencia de la media muestral está en la zona de rechazo")
        print("Conclusión --> Hay suficiente evidencia para rechazar la Hipotesis Nula.  Las muestras provienen de diferentes poblaciones")

  print("-------------------------------------------------------------------")
  print("P-Valor :")
  t_valor2 = d / sqrt((s_a**2)/n_a + (s_b**2)/n_b )
  print("probabilidad acumulada = ",t.cdf(t_valor2,gl))
  p_valor = 2 * (1 - t.cdf(abs(t_valor2), gl))
  #Imprimir información importante
  print("t_valor  = ",t_valor2)
  print("p_valor = ",p_valor)

  if p_valor < significancia:
    print(p_valor,"<",significancia)
    print("Conclusión --> Hay suficiente evidencia para rechazar la Hipotesis Nula. Las muestras provienen de diferentes poblaciones")
  else:
    print(p_valor,">",significancia)
    print("Conclusión --> No hay suficiente evidencia para rechazar la Hipotesis Nula. Las muestras provienen de la misma población")


In [ ]:
prueba_t_student(grupo_a,grupo_b)

Ho = Las muestras vienen de la misma población
Ha = Las muestras provienen de poblaciones diferentes
Medias muestrales de muestra A = 67.7452609474386, muestra B = 65.2667130570193
Tamaños de la muestra A = 50, muestra B = 50
Desviaciónes Estandar de la muestra A = 9.242849378724245, muestra B = 12.983063679735192
Grados de Libertad = 88.5178049116224
x̄1 - x̄2 ~ N(μ^x̄1 - μ^x̄2 = μ1 - μ2,s^x̄1-x̄2 = 2.253842084751468)
-------------------------------------------------------------------
Valor Critico :
significancia para dos colas =  0.025
t_valor =  1.9871278522131859
me =  4.478672381099874
d =  2.478547890419293
intervalo confianza = [-2.0,6.957]
La diferencia de la media muestral está fuera de la zona de rechazo
Conclusión --> No hay suficiente evidencia para rechazar la Hipotesis Nula. Las muestras provienen de la misma población
-------------------------------------------------------------------
P-Valor :
probabilidad acumulada =  0.8627774358093039
t_valor  =  1.0996990016239772


**Ya que los supuestos no se cumplieron trataremos de ver una forma de ajustar los datos y ver si podemos transformarlos para intentar cumplir los supuestos**

Prueba T con los datos transformados

Transformación inversa:
La transformación inversa es útil cuando los datos están sesgados hacia valores más bajos. Puedes aplicar la inversión de tus datos restando cada valor de un número constante o dividiéndolo por un número constante.
python

Como pudimos ver en el Análisis exploratorio, se puede observar que el grupo b esta sesgado, por lo tanto decidimos por esta razón transformar los datos con el método "Transformación inversa".

In [ ]:
# Aplicar transformación inversa
constante = 100
grupo_b_transformado = constante - np.array(grupo_b)

grupo_a_transformado = constante - np.array(grupo_a)

print("Datos originales:", grupo_b)
print("Datos transformados (inversa):", grupo_b_transformado)

Supuestos con los datos transformados

In [ ]:
# Verificar normalidad en el Grupo A
shapiro_test_a = shapiro(grupo_a_transformado)
print("Shapiro-Wilk test para Grupo A:", shapiro_test_a)

# Verificar normalidad en el Grupo B
shapiro_test_b = shapiro(grupo_b_transformado)
print("Shapiro-Wilk test para Grupo B:", shapiro_test_b)

Shapiro-Wilk test para Grupo A: ShapiroResult(statistic=0.9827495217323303, pvalue=0.6722097396850586)
Shapiro-Wilk test para Grupo B: ShapiroResult(statistic=0.9713165163993835, pvalue=0.2616133987903595)


La prueba de Shapiro Wilk sigue diciendonos que los datos se distribuyen normal.

In [ ]:
from scipy.stats import levene

# Aplicar la prueba de Levene para verificar la homogeneidad de varianzas
levene_test = levene(grupo_a, grupo_b_transformado)
print("Prueba de Levene para homogeneidad de varianzas:", levene_test)

Prueba de Levene para homogeneidad de varianzas: LeveneResult(statistic=4.030456271317731, pvalue=0.04743905991523835)


In [ ]:
from scipy.stats import bartlett
# Realizar la prueba de Bartlett
estadistica_bartlett, p_valor_bartlett = bartlett(grupo_a_transformado,grupo_b_transformado)

# Imprimir los resultados
print("Estadística de prueba de Bartlett:", estadistica_bartlett)
print("Valor p de la prueba de Bartlett:", p_valor_bartlett)

# Interpretar los resultados
nivel_significancia = 0.05
if p_valor_bartlett < nivel_significancia:
    print("Las varianzas de los grupos son diferentes (rechazamos H0)")
else:
    print("Las varianzas de los grupos son iguales (no rechazamos H0)")


Estadística de prueba de Bartlett: 5.495874887849253
Valor p de la prueba de Bartlett: 0.019061387916391257
Las varianzas de los grupos son diferentes (rechazamos H0)


Apesar de transformar los datos, la prueba Bartlett nos sigue diciendo que las varinzas son diferentes.

In [ ]:
prueba_t_student(grupo_a_transformado,grupo_b_transformado)

Ho = Las muestras vienen de la misma población
Ha = Las muestras provienen de poblaciones diferentes
Medias muestrales de muestra A = 32.2547390525614, muestra B = 34.73328694298071
Tamaños de la muestra A = 50, muestra B = 50
Desviaciónes Estandar de la muestra A = 9.242849378724245, muestra B = 12.98306367973519
Grados de Libertad = 88.51780491162242
x̄1 - x̄2 ~ N(μ^x̄1 - μ^x̄2 = μ1 - μ2,s^x̄1-x̄2 = 2.253842084751468)
-------------------------------------------------------------------
Valor Critico :
significancia para dos colas =  0.025
t_valor =  1.9871278522131859
me =  4.478672381099874
d =  -2.478547890419314
intervalo confianza = [-6.957,2.0]
La diferencia de la media muestral está fuera de la zona de rechazo
Conclusión --> No hay suficiente evidencia para rechazar la Hipotesis Nula. Las muestras provienen de la misma población
-------------------------------------------------------------------
P-Valor :
probabilidad acumulada =  0.137222564190694
t_valor  =  -1.099699001623986

La Prueba T nos dan los mismos resultados, transformar los datos no nos mostró ningun cambio.